### Test the db

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

#### Connecting to the SQL DB

In [2]:
db_path = str(here("data")) + "/csv_xlsx_sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [3]:
db

In [4]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM buyer_order LIMIT 10;")

sqlite
['buyer_order', 'current_stock']


"[('WFXCOMPANY/OC217.1', 'shikha buyer', 'Cancelled', 'WFX SAMPLE THREAD_2200005_2200005', 'SEW-THREAD-WFXSAMPLETHREAD', 'SEWING TRIMS', 'Trims', 'THREAD', 500.0, '2022-01-04 00:00:00', 1000000.0, 'INR', '2022-01-30', None, None, None), ('116180.1', 'Ashhar Buyer', 'Confirmed', 'WFX SAMPLE STYLE_2200006', 'JAYDEE10001', 'WOVEN', 'Apparel', 'JEANS', 100.0, '2021-11-19 00:00:00', 10000.0, 'INR', '2021-11-19', None, None, None), ('116181.1', 'Ashhar Buyer', 'Confirmed', 'WFX SAMPLE STYLE_2200006', 'JAYDEE10001', 'WOVEN', 'Apparel', 'JEANS', 100.0, '2021-11-19 00:00:00', 1000.0, 'INR', '2021-11-19', None, None, None), ('116184.1', 'Ashhar Buyer', 'Cancelled', 'WFX SAMPLE STYLE_2200006', 'JAYDEE10001', 'WOVEN', 'Apparel', 'JEANS', 100.0, '2021-11-19 00:00:00', 10000.0, 'INR', '2021-11-19', None, None, None), ('116186.1', 'Ashhar Buyer', 'Confirmed', 'WFX SAMPLE STYLE_2200006', 'JAYDEE10001', 'WOVEN', 'Apparel', 'JEANS', 100.0, '2021-11-19 00:00:00', 10000.0, 'INR', '2021-11-19', None, None,

### Test the access to environment variable

In [5]:
from dotenv import load_dotenv
import os
print("Environment variables are loaded:", load_dotenv())
print("test by reading a variable:", os.getenv("OPENAI_API_TYPE"))

Environment variables are loaded: True
test by reading a variable: gemini-2.5-flash


### Test the GPT MODEL

In [6]:
from openai import OpenAI

In [7]:
messages = [
    {"role": "system", "content": str(
        "You are a helpful assistant"
    )},
    {"role": "user", "content": str("hello")}
]

In [8]:
client = OpenAI(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["API_KEY"]
)

In [9]:
response = client.chat.completions.create(
    messages = messages,
    model=os.getenv("MODEL"),
    temperature=0.0,
    
)

In [10]:
print(response.choices[0].message.content)

Hello! How can I help you today?


### 1. SQL Query Chain

In [11]:
## Load the LLM

from langchain_openai import ChatOpenAI

In [12]:
model_name = os.getenv("MODEL")
base_url=os.environ["BASE_URL"]
api_key=os.environ["API_KEY"]

llm = ChatOpenAI(
    model_name=model_name,
    openai_api_key=api_key,
    openai_api_base = base_url,
    temperature=0.0)

In [13]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Identify the buyers with the most orders placed this year."})
print(response)

SQLQuery: SELECT
  T1.buyername,
  COUNT(T1.buyerorderno) AS NumberOfOrders
FROM buyer_order AS T1
WHERE
  STRFTIME('%Y', T1.buyerorderdate) = STRFTIME('%Y', 'now')
GROUP BY
  T1.buyername
ORDER BY
  NumberOfOrders DESC
LIMIT 5;


### Execute the query to make sure it's valid

In [14]:
db.run(response)

OperationalError: (sqlite3.OperationalError) near "SQLQuery": syntax error
[SQL: SQLQuery: SELECT
  T1.buyername,
  COUNT(T1.buyerorderno) AS NumberOfOrders
FROM buyer_order AS T1
WHERE
  STRFTIME('%Y', T1.buyerorderdate) = STRFTIME('%Y', 'now')
GROUP BY
  T1.buyername
ORDER BY
  NumberOfOrders DESC
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [15]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import create_sql_query_chain

sql_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template="""
    You are a SQL expert. 
    Write only a valid SQLite SQL query as output. 
    Do not include markdown, triple backticks, or explanations.
    
    Tables:
    {table_info}
    
    Question:
    {input}
    
    Return at most {top_k} rows if applicable.
    
    SQL:
    """
)

chain = create_sql_query_chain(llm, db, prompt=sql_prompt)

response = chain.invoke({"question": "Identify the buyers with the most orders placed this year."})
print(response)

SELECT buyername
FROM buyer_order
WHERE strftime('%Y', buyerorderdate) = strftime('%Y', 'now')
GROUP BY buyername
ORDER BY COUNT(buyerorderno) DESC
LIMIT 5


In [16]:
db.run(response)

"[('MaX TP Buyer',), ('GEM BUYER',), ('PPC Buyer1',), ('Manjeet',), ('ASMARA DELHI Lands’ End, Inc',)]"

In [17]:
from pyprojroot import here
import sys
import os
project_root = here()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [18]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import create_sql_query_chain
from src.utils.load_config import LoadConfig

Environment variables are loaded: True


In [19]:
## Updated the Yaml file for the sql prompt
APPCFG = LoadConfig()

sql_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=APPCFG.llm_config["sql_generation_prompt"]
)

chain = create_sql_query_chain(llm, db, prompt=sql_prompt)

In [20]:
response = chain.invoke({"question": "Identify the buyers with the most orders placed this year."})
print(response)
db.run(response)


SELECT
  buyername
FROM buyer_order
WHERE
  STRFTIME('%Y', buyerorderdate) = STRFTIME('%Y', 'now')
GROUP BY
  buyername
ORDER BY
  COUNT(buyerorderno) DESC
LIMIT 5;


"[('MaX TP Buyer',), ('GEM BUYER',), ('PPC Buyer1',), ('Manjeet',), ('ASMARA DELHI Lands’ End, Inc',)]"

In [21]:
chain.get_prompts()[0].pretty_print()

You are a SQL expert.
Write only a valid SQLite SQL query as output.
Do not include markdown, triple backticks, or explanations.

Tables:
{table_info}

Question:
{input}

Return at most 5 rows if applicable.

SQL:



### Adding QuerySQLDataBaseTool to the Chain

Execute SQL queries safely

Important: Executing SQL directly is the most sensitive step in creating a SQL chain.

* Carefully assess whether it’s acceptable to allow automated queries on your database.
* Always restrict database permissions to the minimum required (prefer read-only mode).
* For production systems, consider adding a human approval step before executing any query (see below for options).

The `QuerySQLDataBaseTool` allows us to execute queries safely within a chain:

In [22]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import create_sql_query_chain

# Prompt from YAML (no backticks)
sql_prompt = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=APPCFG.llm_config["sql_generation_prompt"]
)

# Create SQL writer & executor
write_query = create_sql_query_chain(llm, db, prompt=sql_prompt)
execute_query = QuerySQLDataBaseTool(db=db)

# Combine into one chain
chain = write_query | execute_query

# Test
result = chain.invoke({"question": "Identify the buyers with the most orders placed this year."})
print(result)


C:\Users\sawan\AppData\Local\Temp\ipykernel_18008\588051697.py:13: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


[('MaX TP Buyer',), ('GEM BUYER',), ('PPC Buyer1',), ('Manjeet',), ('ASMARA DELHI Lands’ End, Inc',)]


### Answer the Question in User Friendly Manner

In [23]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Friendly answer prompt (from YAML for consistency)
answer_prompt = PromptTemplate.from_template(
    APPCFG.llm_config["agent_llm_system_role"]
)

# Answer chain: Pass query + result + question into LLM
answer_chain = answer_prompt | llm | StrOutputParser()

# Full chain: Generate SQL -> Execute -> Pass to answer
chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(result=itemgetter("query") | execute_query)
    | answer_chain
)

# Test with a question
response = chain.invoke({"question": "Who is the Top buyer"})
print(response)


The Top buyer is MaX TP Buyer.


### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions


In [24]:
from langchain_community.agent_toolkits import create_sql_agent

safe_agent_system_prompt = """
You are a helpful SQL assistant.
You are connected to a read-only SQLite database.
You must only generate SELECT queries.
Never use INSERT, UPDATE, DELETE, DROP, or ALTER.
"""

agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="openai-tools",
    verbose=True,
    agent_executor_kwargs={"system_message": safe_agent_system_prompt}
)

# Test the agent
response = agent_executor.invoke({"input": "Show top 5 buyers with most orders"})
print(response)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:  Finally, I will construct the query to get the top 5 buyers with the most orders.


buyer_order, current_stock
Invoking: `sql_db_schema` with `{'table_names': 'buyer_order'}`



CREATE TABLE buyer_order (
	buyerorderno TEXT, 
	buyername TEXT, 
	buyerorderstatus TEXT, 
	stylename TEXT, 
	stylecode TEXT, 
	productgroup TEXT, 
	category TEXT, 
	subcategory TEXT, 
	buyerorderqty REAL, 
	buyerorderdate TEXT, 
	buyerordervalue REAL, 
	currency TEXT, 
	buyerdeliverydate TEXT, 
	buyershippedqty REAL, 
	buyershippedvalue REAL, 
	buyershippedinvoiceno TEXT, 
	PRIMARY KEY (buyerorderno)
)

/*
3 rows from buyer_order table:
buyerorderno	buyername	buyerorderstatus	stylename	stylecode	productgroup	category	subcategory	buyerorderqty	buyerorderdate	buyerordervalue	currency	buyerdeliverydate	buyershippedqty	buyershippedvalue	buyershippedinvoiceno
WFXCOMPANY/OC217.1	shikha buyer	Cancelled	W